# Python Advanced

## Decorator fixSeed

* 寫的很好的 [RealPython tutorial](https://realpython.com/primer-on-python-decorators/#stateful-decorators)

In [181]:
# 固定 seed = 0 版本

import numpy as np
import functools

def fixSeed(fnc):
    @functools.wraps(fnc)
    def wrapperFixSeed(*args, **kargs):
        np.random.seed(0)
        res = fnc(*args, **kargs)
        np.random.seed()
        return res
    return wrapperFixSeed

@fixSeed
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5488135039273248
0.10909977529872539


In [177]:
# 接受 argument 版本，但變成一定要指定 seed

import numpy as np
import functools

def fixSeed(seed=0):
    def decoratorFixSeed(fnc):
        @functools.wraps(fnc)
        def wrapperFixSeed(*args, **kargs):
            np.random.seed(seed)
            res = fnc(*args, **kargs)
            np.random.seed()
            return res
        return wrapperFixSeed
    return decoratorFixSeed

@fixSeed(100)
def printRand():
    print(np.random.uniform())
    
printRand()
print(np.random.uniform())

0.5434049417909654
0.5634875866820428
